<a href="https://colab.research.google.com/github/woukl22/MachineLearningStudy_osamLecture/blob/main/2_MLP(Multi-Layer%20Perception).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Model / data parameters
num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
# x_train의 shape이 이미지라 28, 28이다. 
# fully connected를 할 것이기 때문에 마지막 axis가 하나여야하므로 reshape을 해줘야 한다.

# reshape의 첫번째 argument에는 x_train의 길이를 넣어 기존 x_train의 첫번째 요소와 같게 해준다
# 두 번째 argument는 -1을 넣었는데, -1의 의미는 너가 알아서 계산하라는 뜻이다.
# len(x_train) x ?를 해서 60000 x 28 x 28의 값이 나와야하므로 두 번째 요소에 28*28인 784가 들어가게 된다.
x_train = x_train.reshape((len(x_train), -1))

In [5]:
# 숫자의 범위가 0~1 사이로 normalized된게 학습에 효과가 좋기 때문에
# 정규화를 하기 위해 가장 큰 값인 255로 나눠준다.
# 가장 큰 값이 255인 이유는 위에서 x_train.shape을 해보면 255가 제일 큰 값으로 나온다.
x_train = x_train.astype("float32") / 255

In [6]:
# 28 * 28 = 784
x_train.shape

(60000, 784)

In [7]:
y_train.shape

(60000,)

In [8]:
# convert class vectors to binary class matrices
# 원핫 인코딩의 형태로 바꾸고 싶어서 to_categorical 메소드를 사용한다.
y_train = keras.utils.to_categorical(y_train, num_classes)

In [9]:
y_train.shape

(60000, 10)

In [10]:
# 여기서 나오는 y_train의 shape이 원핫인코딩 형태이다.
# y_train[0]값이 5인데, 5를 [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]로 표현함.
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [11]:
# 모델 디자인을 해보자.
# 현재 input값이 60000 x 784이고,
# output값은 60000 x 10이 나오도록 정해져 있다.
# 우리가 할 것은 가운데에 있는 hidden layer를 정해주는 것이다.
model = keras.Sequential(
    [
     keras.Input(shape=x_train.shape[-1]),
     layers.Dense(256, activation="relu"),
     layers.Dense(128, activation="relu"),
     layers.Dense(num_classes, activation="softmax")
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


dense_1에서 256 x 128의 히든 레이어의 파라미터 수가 32768이 아니라 32896인 이유는<br>
256 * 128 + 128로 bias까지 더해줬기 때문이다!!

그래서 dense_2에서 128 x 10의 행렬도 파라미터 수는 128*10+10을 해서 1290이다.

첫 번째 input layer는 feature size가 784이니까 784x256+256 하면 200960이 나온다.


In [13]:
# 학습을 시켜보자
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=1, validation_split=0.1)

1688/1688 [==============================] - 7s 4ms/step - loss: 0.0926 - accuracy: 0.9715 - val_loss: 0.0855 - val_accuracy: 0.9733


총 6만개의 이미지가 있다.<br>
이 때 32의 batch size를 주는 거니까 32개의 이미지를 넣어가면서 학습을 하겠다는 의미<br>
그럼 60000/32 = 1875이므로 1875번을 반복해야 끝이 난다.<br>
그런데 실제 학습을 한 것을 보면 1688번만 학습을 한 것으로 나온다.<br>
이 이유는 10%는 떼놓고 90%에 대해서만 32번으로 나눠서 학습을 하기 때문에 1688이 나온 것이다.<br>
10% 떼놓은 것은 validation이다.